In [1]:
import torch
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

#torch.cuda.empty_cache()

<weakproxy at 0x7fc60da086d0 to Device at 0x7fc60da0b4f0>

## Reading the transcripts

In [4]:
import pandas as pd
import numpy as np

f = "/home/nikolal/projects/parlaspeech/transcripts_normalized.txt"
df = pd.read_table(f, names=["unknown1", "path", "sentence"],
        #nrows=2000
        )

df["path"] = "/home/peterr/macocu/task6_speech/data/" + \
    df.path.str.replace("flac/", "") + ".wav"
df = df.drop(columns="unknown1")
import os
df["filesize"] = df["path"].apply(os.path.getsize)
for char in ['*', ':', ';']:
    df["sentence"] = df.sentence.str.replace(char, "")
    
df.head()


,path,sentence,filesize
0,/home/peterr/macocu/task6_speech/data/00000001...,poštovane kolegice i kolege zastupnici molio b...,249678
1,/home/peterr/macocu/task6_speech/data/00000002...,prvi klub koji se javio za stanku je klub esde...,118478
2,/home/peterr/macocu/task6_speech/data/00000003...,u njihovo ime predstavnik bojan glavašević,96078
3,/home/peterr/macocu/task6_speech/data/00000004...,hvala gospodine predsjedniče,38478
4,/home/peterr/macocu/task6_speech/data/00000005...,poštovani gospodine predsjedniče hrvatskog sab...,368078


## performing the train_test split

In [6]:
from sklearn.model_selection import train_test_split

common_voice_train_df, common_voice_test_df = train_test_split(df, train_size=0.8, )
# train.loc[:,"split"] = "train"
# test.loc[:,"split"] = "test"

common_voice_train_df.reset_index(drop=True, inplace=True)
common_voice_test_df.reset_index(drop=True, inplace=True)

## Reading the audio file with `datasets.Audio`

In [7]:
import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

common_voice_train.loc[:, "audio"] = common_voice_train.path.apply(load_audio)
common_voice_test.loc[:, "audio"] = common_voice_test.path.apply(load_audio)

common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test)

/home/peterr/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/peterr/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Preparing the training pipeline

In [9]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)


file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
